# *Geração da base dados - apartamentos à venda em Fortaleza - CE / ZAP Imóveis*

In [ ]:
#Chamando bibliotecas
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
#Extraindo o valor de uma página
url_inicial = 'https://www.zapimoveis.com.br/venda/apartamentos/ce+fortaleza/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [ ]:
#Seletores
seletor_card = 'li[data-cy="rp-property-cd"]' 
seletor_preco = 'p[class="text-2-25 text-neutral-120 font-semibold"]'
seletor_tamanho = 'li[data-cy="rp-cardProperty-propertyArea-txt"]'
seletor_quartos = 'li[data-cy="rp-cardProperty-bedroomQuantity-txt"]'
seletor_banheiros = 'li[data-cy="rp-cardProperty-bathroomQuantity-txt"]'
seletor_estacionamento = 'li[data-cy="rp-cardProperty-parkingSpacesQuantity-txt"]'
seletor_bairro = 'h2[class="text-ellipsis text-2 font-semibold overflow-hidden font-secondary"]'
seletor_especial = 'li[class="flex flex-row items-center gap-1 px-1 rounded-1 text-nowrap h-[28px] text-center text-1-75 leading-[28px] [&>svg]:size-2 [&>span]:size-2 shadow-bottom-3 bg-neutral-120 text-neutral-70 justify-center"]'

In [ ]:
servico = Service(ChromeDriverManager().install())
opcoes = webdriver.ChromeOptions()
navegador = webdriver.Chrome(service=servico, options=opcoes)
navegador.get(url_inicial)
time.sleep(5)
try:
    seletor_cookies = 'button[id="adopt-accept-all-button"]'
    botao_cookies = navegador.find_element(By.CSS_SELECTOR,seletor_cookies)
    botao_cookies.click()
    print("Aceitando Cookies")
    time.sleep(3)
except NoSuchElementException:
    print("Não houve necessidade de aceitar cookies")
pagina_atual =1
lista_imoveis = []
while True:
    print("Abrindo página")
    time.sleep(5)
    html_completo = navegador.page_source
    soup = BeautifulSoup(html_completo, 'html.parser')
    todos_os_anuncios = soup.select(seletor_card)
    #Função de pegar as informações da página
    for anuncio in todos_os_anuncios:
        preco = anuncio.select_one(seletor_preco)
        if preco:
            preco = preco.get_text(strip=True)
        else:
            preco = "Valor não encontrado"
        tamanho = anuncio.select_one(seletor_tamanho)
        if tamanho:
            tamanho = tamanho.get_text(strip=True)
        else:
            tamanho = "Valor não encontrado"
        quartos = anuncio.select_one(seletor_quartos)
        if quartos:
            quartos = quartos.get_text(strip=True)
        else:
            quartos = "Valor não encontrado"
        banheiros = anuncio.select_one(seletor_banheiros)
        if banheiros:
            banheiros = banheiros.get_text(strip=True)
        else:
            banheiros = "Valor não encontrado"
        vagas = anuncio.select_one(seletor_estacionamento)
        if vagas:
            vagas = vagas.get_text(strip=True)
        else:
            vagas= "Valor não encontrado"
        bairro = anuncio.select_one(seletor_bairro)
        if bairro:
            bairro = bairro.get_text(strip=True)
        else:
            bairro = "Valor não encontrado"
        dados = {'Preco':preco,
                'Tamanho':tamanho,
                'Quartos':quartos,
                'Banheiros':banheiros,
                'Vagas':vagas,
                'Bairro':bairro
                }
        lista_imoveis.append(dados)
    #Apertar o botação de cookies e próximo
    try:
        seletor_botao = 'button[data-testid="next-page"]'
        botao_proxima = navegador.find_element(By.CSS_SELECTOR,seletor_botao)
        if botao_proxima.is_enabled():
            navegador.execute_script("arguments[0].scrollIntoView(true)",botao_proxima)
            time.sleep(2)
            print("Pagina rolada")
            navegador.execute_script("arguments[0].click()",botao_proxima)
            pagina_atual+=1
        else:
            print("Fim da paginação")
            break
    except NoSuchElementException:
        print("Fim da paginação 2")
        break
navegador.quit()
print("Navegar encerrado")

In [ ]:
print(len(lista_imoveis))

In [ ]:
print(pagina_atual)

In [ ]:
df = pd.DataFrame(lista_imoveis)
df

In [ ]:
#Salvando em arquivo CSV
nome_do_arquivo = 'imoveis_zap_fortaleza.csv'
df.to_csv(nome_do_arquivo, index=False, sep=';', encoding='utf-8-sig')